In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.signal import butter, lfilter
import os
from HelperFunctions import *

import matplotlib.pyplot as plt
%matplotlib notebook


In [5]:
base_path = os.getcwd()
data_path = os.path.join(base_path, 'Data')
data_files = os.listdir(data_path)

## Here We need to add the following:
1. Generating the integral action data
2. creating the state vector x ={distance,distance_dot, current, current integral}
    The output vector is the pwm for each motor- PWM1, PWM2, PWM3, PWM4
3. Need to filter all the data with the same HPF configuration
4. Initializing stabalizing controller


## Questions-
1. Butterworth filter or regular 1st order HPF?
2. How we utilize the fact that we have all the PWM values and that they are different? maybe remove the pid addition and take only the throttle?


# The Algorithm Starts here:

Initializing the variables:

In [ ]:

kron_xx = np.empty((0, np.size(x,0)**2))
kron_xu = np.empty((0, np.size(x,0)*np.size(u,0)))
delta_x = np.empty((0, np.size(x,0)**2))
Ixx = np.empty((0, np.size(x,0)**2)) # Initialize as an empty 2D array with shape (0, 2)
Ixu = np.empty((0, np.size(x,0)))

l = np.size(x,0)*(np.size(x,0)+1)//2+np.size(x,0)*np.size(u,0) # u is scalar so m=1. we have 2 states, therefore 2 equations. Q is 2x2. defining l this way will help us to set the window of the integrations in order to get a valid dimensions for Ixx and Ixu
iterations = 1669  # Number of time steps

intr = int(iterations//(l +1)) # defining the interval for the integrations. this interval will give us Ixx and Ixu with the ammount of equations needed. In our case 2 equations.

Generating the Variables needed for the Algorithm

In [ ]:
for i in range(intr,iterations, intr):
    x_i = x[:,i]
    x_i_1 = x[:,i-intr]
    delta_xx_i = np.kron(x_i, x_i) - np.kron(x_i_1, x_i_1)
    delta_x = np.vstack((delta_x, [delta_xx_i]))

    
for i in range(0,iterations):
    x_i = x[:,i]
    u_i = u[i]
    kron_xx_i = np.kron(x_i, x_i)
    kron_xu_i = np.kron(x_i, u_i)
    kron_xx = np.vstack((kron_xx, [kron_xx_i]))
    kron_xu = np.vstack((kron_xu, [kron_xu_i]))

for i in range(intr,iterations, intr):
    window = slice(max(0, i-intr), i+1)  # This defines the time steps t0 to t1 etc.
    new_Ixu = np.trapz(kron_xu[window,:], dt[window], axis = 0)
    new_Ixx = np.trapz(kron_xx[window,:], dt[window], axis =0)
    Ixu = np.vstack((Ixu, [new_Ixu]))
    Ixx = np.vstack((Ixx, [new_Ixx]))

# The actual learning algorithm

In [ ]:
n = np.size(x,0)
m = 1
Q = np.array([[100, 0], [0, 100]])
R_ = np.array([[1]])
K_i = K_lqr # starting with a stabilizing control value
N_ = create_N_matrix(n)

K_adp = np.empty((0, np.size(x,0)))
P_adp = []
P_i_old = np.zeros_like(Q)

norm_Pi = np.array([])

for i in range(1,iterations):
    # Calculating Q_i:
    Q_i = Q + K_i.T @ R_ @ K_i
    
    # Calculating Y_i:
    Y_i = Ixx @ vec(Q_i) # Shai added - in the begining
    X_i = np.hstack((-delta_x @ N_, 2* Ixx@np.kron(np.eye(n), K_i.T @ R_) + 2*Ixu @ np.kron(np.eye(n), R_)))
    
    # Calculating Theta_i:
    Theta_i = np.linalg.lstsq(X_i, Y_i, rcond=None)[0]
    # Theta_i = np.linalg.inv(X_i.T @ X_i) @ X_i.T @ Y_i  # Using the normal equation for least squares
    
    
    svecP_i = Theta_i[:(n**2-n*m + 1)]
    P_i = svec2matrix(svecP_i)
    norm_Pi = np.append(norm_Pi, np.linalg.norm(P_i))
    
    K_i = Theta_i[(n**2-2*m+1):].reshape(K_lqr.shape)

    
    K_adp = np.vstack((K_adp, K_i))
    P_adp.append(P_i)
    
    if (abs(np.linalg.norm(P_i_old - P_i)) < 0.005) and (np.all(np.linalg.eigvals(P_i) > 0)):
        print('Converged')
        
        break
    
    P_i_old = P_i